# <u>***This notebook assumes you have already created a DLC project folder, labeled frames, and uploaded this project folder to your Google Drive already.***</u>
You can find all the docs how to do this here: https://deeplabcut.github.io/DeepLabCut

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628180434489-T0RIWEJJU0FJVOT6FNVD/maDLC.png?format=200w)

# **This notebook demonstrates how to run DeepLabCut with <u>Google Colab</u> to train a model and analyze videos for a single animal tracking project.**

- Create a single-animal training set
- Train a network
- Analyze videos
- Analyze new videos

*This notebook is loosely based off of the outdated multi-animal GoogleColab tutorial notebook on the DeepLabCut website. It has been edited and updated with fixes to make it work.*

# <u> **BEFORE RUNNING ANY CELLS BELOW** </u>

# <u> **Step 1. Make video subfolder containing your videos.** </u>

### ***Move all your videos in your DeepLabCut project folder to their own folder within this videos folder. Call this folder whatever you want as we will use this name later. A suggestion is "Run1"***

### If you chose "Run1" for folder name the resulting folder containing all your videos would be "videos/Run1/" within your project folder on Google Drive.

# <u> **Step 2. Connect to GPU runtime in Google Colab** </u>
## **In the toolbar above click "Connect GPU"**

# <u> **Step 3. Install DeepLabCut** </u>

### **This next code cell below installs all the dependencies needed to run DeepLabCut in Google Colab as well as DeepLabCut itself (no GUI available in Colab).**

### ***This takes about 15-20 minutes to install and set up everything before any other cells below can be ran***

In [ ]:
# Upgrade python package manager
!pip3 install --upgrade pip

# Install deeplabcut
!pip3 install -qqq deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

# Install deeplabcut google colab specific things
!pip3 install -qqq --upgrade scikit-image
!pip3 install -qqq pickle5
!pip3 install -qqq torch

# this is so the next line when installing updated cuda doesn't ask for keyboard interaction
!DEBIAN_FRONTEND=noninteractive apt-get install keyboard-configuration

# Update cuda to latest version
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb
!sudo apt-get update
!sudo apt-get -y install cuda

# This is just to check what version of cuda we have now to make sure it installed correctly
#!nvcc --version

# Add newly installed cuda version to PATH
!export PATH=/usr/local/cuda-12.4/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-12.4/include:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI/lib64

# Downgrade tensorflow to older version otherwise deeplabcut.train_network() won't work later
!pip3 uninstall -y -qqq tensorflow
!pip3 install -qqq tensorflow==2.8.0

# Install other dependency packages
!pip3 install -qqq segmentation-models
!pip3 install -qqq pycocotools
!pip3 install -qqq opencv-python-headless

# <u> **Step 4. Link Google Drive to Google Colab** </u>

## **Now, let's link to your Google Drive containing your DeepLabCut project folder. Run this next cell and follow the authorization instructions.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

# <u> **Step 5. Import DeepLabCut and other python modules we'll need.** </u>

In [ ]:
import deeplabcut
import os
import shutil

# <u> **Step 6. Edit project_folder path to location in your Google Drive** </u>

# **This is the <u> first</u> cell you will need to edit to fit your project**

## The base path for Google Drive is "/content/drive/MyDrive/"

## In this example my project folder "myprojectfolder" is in a subfolder "DeepLabCut" in my Google Drive so that's why I added "DeepLabCut/" to this.


In [ ]:
# set project_folder variable to full path location of project folder
project_folder = '/content/drive/MyDrive/DeepLabCut/myprojectfolder/'

## ***If you get an error from running the next cell below then you did not set the project_folder location correctly in the previous cell above.***

In [ ]:
# move to directory where project folder is located
os.chdir(project_folder)

# verify you are in the correct location
os.getcwd()

# <u> **Step 7. Edit this_phase variable** </u>

# **This is the <u>second</u> cell you will need to edit to fit your project**

## ***This is where we'll use the name of the video subfolder you created in <u> Step 1. </u>***

## **For example, if you used "Run1" for the video subfolder name then you would set 'this_phase' to be "Run1" like below**

In [ ]:
# Enter phase for these videos for storing analysis output data later
this_phase = 'Run1'

# <u> **Step 8. Create variables and folders for our output later** </u>

In [ ]:
path_config_file = project_folder + 'config.yaml'

# Enter the folder of videos to analyze.
videofile_path = [project_folder + 'videos/' + this_phase]

# Which shuffle do you want to create and train? Edit if needed, 1 is the default.
shuffle = 1

# Create a results folder within project directory to store output data
if not os.path.exists("results"):
    os.mkdir('results')

# Create location in results folder where output data will be stored
analysis_destination_path = project_folder + 'results/' + this_phase

if not os.path.exists(analysis_destination_path):
    os.mkdir(analysis_destination_path)

# <u> **Step 9. Create training dataset** </u>

## ***All options for net_type and augmenter_type if you do not want the options set below:***

### net_type_list = ['resnet_50', 'resnet_101', 'resnet_152', 'mobilenet_v2_1.0', 'efficientnet-b0', 'efficientnet-b1', 'efficientnet-b2', 'efficientnet-b3', 'efficientnet-b4', 'efficientnet-b5', 'efficientnet-b6']

### augmenter_type_list = ['default', 'scalecrop', 'imgaug', 'tensorpack', 'deterministic']

In [ ]:
deeplabcut.create_training_dataset(path_config_file,
                                   net_type ='resnet_50',
                                   augmenter_type = 'imgaug',
                                   Shuffles = [shuffle])

### - **Useful information for deeplabcut.create_training_dataset()**

***Do not*** run this a <u>**second**</u> time if only increasing max_iters for deeplabcut.train_network(). Only rerun this line again if you edit the config to add more videos, extract more frames, and labeled more frames.


This function trains the network for a specific shuffle of the training dataset.
 - more info: https://deeplabcut.github.io/DeepLabCut/docs/

# <u> **Step 10. Start training** </u>

## **This is the step that *will* take the longest. At least a few hours depending on how many videos / frames you labeled and maxiters set below.**

## <u> *You MUST leave the browser open on the computer this is running on OR after running the cell below, you can open up this notebook simultaneously on the Google Colab site on your phone or another device and just check the status occasionally.* </u>

## ***This will make sure your session remains connected as Google Colab <u>will</u> timeout if you close the browser, and <u>you will have to start over</u>. You can reopen the notebook and continue the Google Colab session on any computer as long as you still have it open and connected on your phone or another device.***

*Note: Google Colab time limits are not clearly defined but you can probably assume anything running longer than 12 hours will be automatically disconnected as this notebook assumes you are using the free tier and Google Colab prioritizes its paying subscribers.

For a rough idea of time, my training using 30 frames labeled for four 5 minute videos with the maxiters set to 100000 usually completes in around 8 hours, although your experience may vary.

In [ ]:
deeplabcut.train_network(path_config_file,
                         saveiters = 10000,
                         displayiters = 1000,
                         maxiters = 100000)

### **The above cell will probably show an error when it is done with all iterations. This is totally normal and can be disregarded.**

### - **Useful information for deeplabcut.train_network()**

When training for a <u>**second**</u> time, **if** the 'max_iters' variable is set to the same value as the last snapshot iteration number, this will cause an error. This happens because it actually now starts from the iteration of the last snapshot.

For example, if you originally trained it with 100,000 iterations and now want another 100k iterations to improve accuracy, then change max_iters to 200000

# <u> **Step 11. Analyze videos with now trained model.** </u>

## **This uses the now trained model to analyze all the videos in your this_phase video subfolder.**

In [ ]:
deeplabcut.analyze_videos(path_config_file,
                          videofile_path,
                          shuffle = shuffle,
                          save_as_csv = True,
                          destfolder = analysis_destination_path,
                          videotype = 'mp4')

# <u> **Step 12. Create labeled videos from analyzed files.** </u>

## **This creates labeled videos to visually inspect how training did at labeling and tracking animal.**

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path,
                                draw_skeleton = True,
                                skeleton_color = "white",
                                color_by = "individual",
                                destfolder = analysis_destination_path,
                                videotype = 'mp4',
                                save_frames = False)

# <u> **DONE** </u>

## **Look at the results in the project folder 'results' folder and the labeled videos in your video subfolder in Google Drive to see how training and analysis parameters turned out.**

# <u> **Optional Step. Move files to dedicated folders in results folder** </u>

## ***This is <u> not </u> necessary to run and if you plan to do post-processing steps like manually fixing labels then <u> you will need to move all these files back </u> to videos folder or set their paths manually via command line.***

## <u>**BUT**</u> **if you would like to move all your results to neat dedicated folders based on file type then run this cell.**

In [ ]:
# Create location for labeled-videos
labeled_videos_folder = project_folder + 'labeled-videos/'

if not os.path.exists(labeled_videos_folder):
    os.mkdir(labeled_videos_folder)

# Organize output data into folders by file type
os.chdir(analysis_destination_path)
these_labeled_videos = labeled_videos_folder + this_phase

if not os.path.exists('CSVs'):
    os.mkdir('CSVs')
if not os.path.exists('h5'):
    os.mkdir('h5')
if not os.path.exists('pickle'):
    os.mkdir('pickle')
if not os.path.exists(these_labeled_videos):
    os.mkdir(these_labeled_videos)

all_analysis_files = os.listdir()

for f in all_analysis_files:
    if f.endswith('.csv'):
        shutil.move(f, 'CSVs')
    elif f.endswith('.h5'):
        shutil.move(f, 'h5')
    elif f.endswith('.pickle'):
        shutil.move(f, 'pickle')
    elif f.endswith('.mp4'):
        shutil.move(f, these_labeled_videos)

# **If you <u> *did*</u> choose to run this cell above**
### **Then the results and labeled videos are stored in the results and labeled_videos folders within the project_folder in Google Drive. Look at these to see how training and analysis parameters turned out.**

# <u>**Analyzing New Videos</u>**

## - **When returning to analyze new similar looking videos using this already trained model, re-run Steps 1-6**

## - **Then after re-running Steps 1-6 set 'this_phase' in the cell below to the name of the new subfolder of videos within the videos folder you now want to analyze. This should be the <u> only</u> variable you need to change and is the name of the new folder within videos folder, for example 'Run2'**

## - **Do <u> NOT</u> retrain model (Steps 7-10) <u> unless</u> your labeled videos do not look accurate from first training model.**

In [ ]:
this_phase = 'Run2'

In [ ]:
new_videofile_path = [project_folder + 'videos/' + this_phase]

analysis_destination_path = project_folder + 'results/' + this_phase

if not os.path.exists(analysis_destination_path):
    os.mkdir(analysis_destination_path)

In [ ]:
print("Analyzing new videos")

deeplabcut.analyze_videos(path_config_file,
                          new_videofile_path,
                          shuffle = shuffle,
                          save_as_csv = True,
                          destfolder = analysis_destination_path,
                          videotype = 'mp4')

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                new_videofile_path,
                                draw_skeleton = True,
                                skeleton_color = "white",
                                color_by="individual",
                                destfolder = analysis_destination_path,
                                videotype = 'mp4',
                                save_frames = False)

# **Optional Step. Move files to dedicated folders in results folder**

In [ ]:
# Set path for labeled-videos folder
labeled_videos_folder = project_folder + 'labeled-videos/'

# Organize output data into folders by file type
os.chdir(analysis_destination_path)
these_labeled_videos = labeled_videos_folder + this_phase

if not os.path.exists('CSVs'):
    os.mkdir('CSVs')
if not os.path.exists('h5'):
    os.mkdir('h5')
if not os.path.exists('pickle'):
    os.mkdir('pickle')
if not os.path.exists(these_labeled_videos):
    os.mkdir(these_labeled_videos)

all_analysis_files = os.listdir()

for f in all_analysis_files:
    if f.endswith('.csv'):
        shutil.move(f, 'CSVs')
    elif f.endswith('.h5'):
        shutil.move(f, 'h5')
    elif f.endswith('.pickle'):
        shutil.move(f, 'pickle')
    elif f.endswith('.mp4'):
        shutil.move(f, these_labeled_videos)